In [1]:
import pandas as pd

In [2]:
#Reading dataset
data = pd.read_csv('/train.csv')

In [3]:
#We will check the shape of the dataset and the top five elements of the dataset.
data.shape

(1157, 5)

In [4]:
#Head of the dataset
data.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
2,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1
3,z13lfzdo5vmdi1cm123te5uz2mqig1brz04,ferleck ferles,2013-11-27T21:39:24,Subscribe to my channel ﻿,1
4,z12avveb4xqiirsix04chxviiljryduwxg0,BeBe Burkey,2013-11-28T16:30:13,and u should.d check my channel and tell me wh...,1


In [5]:
data.describe()

,CLASS
count,1157.000000
mean,0.506482
std,0.500174
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [6]:
data.isna().sum()

COMMENT_ID      0
AUTHOR          0
DATE          138
CONTENT         0
CLASS           0
dtype: int64

In [7]:
#Fill NaN values with mode values
data["DATE"] = data["DATE"].fillna(data["DATE"].mode())

data.isna().sum()

COMMENT_ID    0
AUTHOR        0
DATE          0
CONTENT       0
CLASS         0
dtype: int64

In [8]:
df=data[['CONTENT','CLASS']]
df.head()

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
2,watch?v=vtaRGgvGtWQ Check this out .﻿,1
3,Subscribe to my channel ﻿,1
4,and u should.d check my channel and tell me wh...,1


In [9]:
## Get the Independent Features

X=df.drop('CLASS',axis=1)

In [10]:
## Get the Dependent features
y=df['CLASS']

In [11]:
y.value_counts()

1    586
0    571
Name: CLASS, dtype: int64

In [12]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [14]:
voc_size=5000

In [15]:
message = X.copy()

In [16]:
message['CONTENT'][1]

'me shaking my sexy ass on my channel enjoy ^_^ \ufeff'

In [17]:
message.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['CONTENT'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
corpus[1]

'shake sexi ass channel enjoy'

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1]

[4493, 4213, 6, 4486, 4924]

In [22]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2834 4486 2592]
 [   0    0    0 ...    6 4486 4924]
 [   0    0    0 ... 4138 4206  158]
 ...
 [   0    0    0 ... 2151 1654  802]
 [   0    0    0 ...    0    0 2468]
 [   0    0    0 ... 3795 2504 2463]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 3022, 2429,  158, 2834, 4486, 2592], dtype=int32)

In [24]:
## Creating model
embedding_vector_features=50
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Dropout(0.25))
model1.add(Bidirectional(LSTM(100))) ##Just add bidirectional!!, except it would just behave as normal LSTM Model
model1.add(Dropout(0.25))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(64,activation='relu'))
model1.add(Dropout(0.25))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 50)            250000    
_________________________________________________________________
dropout (Dropout)            (None, 40, 50)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               120800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                6432      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0

In [25]:
len(embedded_docs),y.shape

(1157, (1157,))

In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
X_final[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 4493, 4213,    6, 4486, 4924], dtype=int32)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=32)

In [29]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=25,batch_size=64)

Epoch 1/25
14/14 [==============================] - 6s 164ms/step - loss: 0.6883 - accuracy: 0.5802 - val_loss: 0.6732 - val_accuracy: 0.6207
Epoch 2/25
14/14 [==============================] - 1s 99ms/step - loss: 0.6396 - accuracy: 0.6805 - val_loss: 0.5901 - val_accuracy: 0.6897
Epoch 3/25
14/14 [==============================] - 1s 100ms/step - loss: 0.4982 - accuracy: 0.7762 - val_loss: 0.4310 - val_accuracy: 0.8310
Epoch 4/25
14/14 [==============================] - 1s 101ms/step - loss: 0.3086 - accuracy: 0.8870 - val_loss: 0.2819 - val_accuracy: 0.8793
Epoch 5/25
14/14 [==============================] - 1s 96ms/step - loss: 0.1604 - accuracy: 0.9516 - val_loss: 0.2022 - val_accuracy: 0.9207
Epoch 6/25
14/14 [==============================] - 1s 97ms/step - loss: 0.0877 - accuracy: 0.9689 - val_loss: 0.1665 - val_accuracy: 0.9276
Epoch 7/25
14/14 [==============================] - 1s 99ms/step - loss: 0.0432 - accuracy: 0.9885 - val_loss: 0.1710 - val_accuracy: 0.9345
Epoch 8/25

In [30]:
y_pred=model1.predict(X_test)

In [31]:
y_test[0:5]

array([0, 0, 1, 1, 1])

In [32]:
y_pred[0:5]

array([[3.70466709e-03],
       [1.06304564e-04],
       [9.90338981e-01],
       [9.99973416e-01],
       [9.99997139e-01]], dtype=float32)

In [33]:
type(y_test), type(y_pred)

(numpy.ndarray, numpy.ndarray)

In [34]:
y_pred.round()[0:5]

array([[0.],
       [0.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred.round())

array([[134,  16],
       [  9, 131]])

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred.round(decimals=0))

0.9137931034482759

In [37]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred.round(decimals=0)))

              precision    recall  f1-score   support

           0       0.94      0.89      0.91       150
           1       0.89      0.94      0.91       140

    accuracy                           0.91       290
   macro avg       0.91      0.91      0.91       290
weighted avg       0.91      0.91      0.91       290



In [38]:
#Reading dataset
df_test = pd.read_csv('/test.csv')

In [39]:
#We will check the shape of the dataset and the top five elements of the dataset.
df_test.shape

(799, 5)

In [40]:
#Head of the dataset
df_test.head()

,ID,COMMENT_ID,AUTHOR,DATE,CONTENT
0,0,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...
1,1,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com
2,2,LZQPQhLyRh9-wNRtlZDM90f1k0BrdVdJyN_YsaSwfxc,Jason Haddad,2013-11-26T02:55:11,"Hey, check out my new website!! This site is a..."
3,3,z122wfnzgt30fhubn04cdn3xfx2mxzngsl40k,Bob Kanowski,2013-11-28T12:33:27,i turned it on mute as soon is i came on i jus...
4,4,z13ttt1jcraqexk2o234ghbgzxymz1zzi04,Cony,2013-11-28T16:01:47,You should check my channel for Funny VIDEOS!!﻿


In [41]:
df_test1=df_test[['ID','CONTENT']]

In [42]:
df_test1.head()

,ID,CONTENT
0,0,Hey guys check out my new channel and our firs...
1,1,just for test I have to say murdev.com
2,2,"Hey, check out my new website!! This site is a..."
3,3,i turned it on mute as soon is i came on i jus...
4,4,You should check my channel for Funny VIDEOS!!﻿


In [43]:
df_test1.isna().sum()

ID         0
CONTENT    0
dtype: int64

In [44]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(df_test1)):
    review = re.sub('[^a-zA-Z]', ' ', df_test1['CONTENT'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [45]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1]

[4245, 3390, 4017, 1853]

In [46]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1284  963 1823]
 [   0    0    0 ... 3390 4017 1853]
 [   0    0    0 ... 4714 1290 1853]
 ...
 [   0    0    0 ... 2395 3284 3795]
 [   0    0    0 ... 4752 2599 3971]
 [   0    0    0 ... 3795 3999  130]]


In [47]:
embedded_docs.shape

(799, 40)

In [48]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       2723, 4348,  158, 1217, 4486, 2415, 2649, 3971, 4627, 4627, 4638,
         52,  963, 1882, 2940, 1284,  963, 1823], dtype=int32)

In [49]:
len(sorted(model1.predict(embedded_docs[0]), reverse=True))


40

In [50]:
corpus = []
sLength = len(df_test1['ID'])
df_test1['CLASS'] = 0 #pd.Series(np.random.randn(sLength), index=df_test1.index)
for i in range(0, len(df_test1)-1):
    review = re.sub('[^a-zA-Z]', ' ', df_test1['CONTENT'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    #corpus.append(review)
    onehot_repr1=[one_hot(words,voc_size)for words in review] 
    embedded_docs1=pad_sequences(onehot_repr1,padding='pre',maxlen=sent_length)
    if (len(embedded_docs1) > 0):
      if ((model1.predict(embedded_docs1)).all() > 0.95):
        df_test1['CLASS'][i] = 1
      else:
        df_test1['CLASS'][i] = 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [55]:
df_test1.head(20)

,ID,CONTENT,CLASS
0,0,Hey guys check out my new channel and our firs...,1
1,1,just for test I have to say murdev.com,1
2,2,"Hey, check out my new website!! This site is a...",1
3,3,i turned it on mute as soon is i came on i jus...,1
4,4,You should check my channel for Funny VIDEOS!!﻿,1
5,5,Hey subscribe to me﻿,1
6,6,subscribe like comment﻿,1
7,7,http://www.ebay.com/itm/171183229277?ssPageNam...,1
8,8,http://ubuntuone.com/40beUutVu2ZKxK4uTgPZ8K﻿,1
9,9,We are an EDM apparel company dedicated to bri...,1


In [52]:
df_test1['CLASS'].value_counts()

1    789
0     10
Name: CLASS, dtype: int64

In [53]:
df2=df_test1[['ID','CLASS']]
df2.head()

,ID,CLASS
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [54]:
# saving the dataframe

df2.to_csv('/file1.csv', index=False)